# flats-in-cracow data wrangling

## Imports

In [78]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from collections import Counter
from IPython.display import display
from sklearn.impute import KNNImputer
from pylab import rcParams
from pathlib import Path

## Setup

In [79]:
# Create directory for images 
Path("img").mkdir(parents=True, exist_ok=True)

# Set default figure size
rcParams['figure.figsize'] = (4, 4)

# Tell pandas how to display floats
pd.options.display.float_format = "{:,.2f}".format

## Goal

I scraped listings of properties for sale in Cracow. We would like to create a model to predict flat prices.

## Data source

Data has been scraped from a website with listings. The data has undergone small transformations along the way. The goal of these transformations was to get the data into a usable state not to check it's validity.

## Data loading

In [80]:
path = '../flats-data/raw_data.csv'

In [81]:
data = pd.read_csv(path, lineterminator='\n')

In [82]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43453 entries, 0 to 43452
Data columns (total 24 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         43336 non-null  object 
 1   City         31948 non-null  object 
 2   District     23668 non-null  object 
 3   Amount       43296 non-null  float64
 4   Currency     43296 non-null  object 
 5   Property     43022 non-null  object 
 6   Seller       43211 non-null  object 
 7   Area         43088 non-null  float64
 8   Rooms        42578 non-null  float64
 9   Bathrooms    27443 non-null  float64
 10  Parking      18399 non-null  object 
 11  Garden       43453 non-null  bool   
 12  Balcony      43453 non-null  bool   
 13  Terrace      43453 non-null  bool   
 14  Floor        43453 non-null  bool   
 15  New          43453 non-null  bool   
 16  Estate       43453 non-null  bool   
 17  Townhouse    43453 non-null  bool   
 18  Apartment    43453 non-null  bool   
 19  Land

First we sort the data in from newest to oldest, forcing rows with missing `Date` values to be last.

In [83]:
data = data.sort_values(by='Date', 
                        ascending=False, 
                        na_position='last', 
                        ignore_index=True)

Next we assume that the `Title` column uniquely identifies a listing.

In [84]:
data = data.drop_duplicates(['Title'], keep='first')

After this the shape of the data is:

In [85]:
print(data.shape)

(8079, 24)


## Data exploration

We check for missing values that we will have to deal with.

In [86]:
missing = data.isnull().sum(axis=0)
missing.name = 'Missing'
missing = missing.to_frame()
missing = missing[missing['Missing'] > 0]
missing.sort_values('Missing', ascending=False)

,Missing
Parking,5133
District,3365
Bathrooms,3356
City,1441
Description,1332
Rooms,199
Area,107
Seller,69
Property,68
Amount,6


### Check numeric columns

We see that we have 24 columns at our disposal. 
We inspect the numeric columns to see what we are dealing with. 
In the `Amount` column we note there is a property for sale that costs 1PLN, clearly a erroneous value. 
Next we note that the enourmous maximum in the `Amount` column. That is quite a lot of money and could be considered a potential outlier.
The maximum and minimum of the `Area` column also indicate the existance of outliers. These values are clearly too large. The data will need to undergo a filtering process.

In [87]:
data.describe()

,Amount,Area,Rooms,Bathrooms
count,"8,073.00","7,972.00","7,880.00","4,723.00"
mean,"675,942.46",147.69,2.90,1.33
std,"730,859.78","4,059.89",1.32,0.64
min,100.00,1.00,1.00,1.00
25%,"395,000.00",43.00,2.00,1.00
50%,"499,900.00",56.00,3.00,1.00
75%,"720,000.00",80.00,4.00,2.00
max,"22,000,000.00","320,000.00",6.00,4.00


### Check binary columns

We inspect the data to see if binary columns are properly populated and check for imbalances.

In [88]:
binary = data.select_dtypes(bool).columns.to_list()

for col in binary:
    tmp = data[[col, 'Amount']]
    tmp = tmp.fillna('NaN')
    tmp = tmp.groupby(col, as_index=False)
    tmp = tmp.count()
    tmp = tmp.rename(columns={'Amount': 'Count'})
    tmp = tmp.sort_values('Count', ascending=False)
    tmp = tmp.reset_index(drop=True)
    display(tmp)

,Garden,Count
0,False,6500
1,True,1579


,Balcony,Count
0,False,5269
1,True,2810


,Terrace,Count
0,False,7135
1,True,944


,Floor,Count
0,False,4905
1,True,3174


,New,Count
0,False,5413
1,True,2666


,Estate,Count
0,False,6893
1,True,1186


,Townhouse,Count
0,False,7350
1,True,729


,Apartment,Count
0,False,6920
1,True,1159


,Land,Count
0,False,6258
1,True,1821


,Studio,Count
0,False,7563
1,True,516


### Check categorical columns

We inspect categorical columns to assert that they contain "valid" values. Most of these columns were generated by a script during the scraping and etl phase of the project.

In [89]:
categorical = data.select_dtypes('object').columns
categorical = categorical.to_list()
omit = ['Title', 'Link', 'Description', 'Date']

for col in categorical:
    if col not in omit:
        tmp = data[['Amount', col]].copy()
        tmp = tmp.fillna('NaN')
        tmp = tmp.groupby(col, as_index=False)
        tmp = tmp.count()
        tmp = tmp.rename(columns={'Amount': 'Count'})
        tmp = tmp.sort_values('Count', ascending=False)
        tmp = tmp.reset_index(drop=True)
        display(tmp)

,City,Count
0,kraków,6638
1,NaN,1441


,District,Count
0,NaN,3365
1,krowodrza,637
2,stare miasto,541
3,podgorze,523
4,nowa huta,375
5,debniki,340
6,bronowice,332
7,pradnik bialy,309
8,pradnik czerwony,248
9,biezanow,247


,Currency,Count
0,pln,8073
1,NaN,6


,Property,Count
0,flat,6938
1,house,1073
2,NaN,68


,Seller,Count
0,realtor,7426
1,owner,584
2,NaN,69


,Parking,Count
0,NaN,5133
1,street,1191
2,garage,1098
3,no parking,506
4,covered,151


### Check text columns

We search for keywords in the data.

In [90]:
# text = data[data['Description'].isna() == False].copy()
# text = text['Description'].to_list()
# text = ' '.join(text)
# text = text.split(' ')
# text = [x for x in text if x.isalpha()]
# text = [x for x in text if len(x) > 3]

In [91]:
# for i in range(5, len(text)-5):
#     if 'piętro' in text[i]:    
#         s = text[i-5:i+5]
#         s = ' '.join(s)
#         print(s)

## Data cleaning

We assume that if we know the district, the `City` is `kraków`.

In [92]:
mask = (data['City'].isna() == True) & (data['District'].isna() == False)
data.loc[mask, 'City'] = 'kraków'

We extract more `Parking` information from the property description.

In [93]:
def extract_parking(x):
    if ('garaż' in x or 'garaz' in x or 'parking' in x) and 'podziemny' in x:
        return 'covered'
    elif ('garaż' in x or 'garaz' in x) and 'podziemny' not in x:
        return 'garage'
    elif 'parking' in x and 'podziemny' not in x:
        return 'street'
    else:
        return 'no parking'

In [94]:
mask = (data['Parking'].isna() == True) & (data['Description'].isna() == False)
data.loc[mask, ['Parking', 'Description']] = data.loc[mask, 'Description'].apply(extract_parking)

In [95]:
mask = data['Parking'].isna() == True
data.loc[mask, 'Parking'] = 'no parking'

We confirm that we have dealt with all the `NaN`s in the `Parking` column.

In [96]:
print(data['Parking'].isna().sum())

0


### Filtering

Next we filter the data according to these rules:

In [97]:
data = data[data['City'] == 'kraków']
data = data[data['Currency'] == 'pln']
data = data[data['Property'] == 'flat']
data = data[(data['Amount'] >= data['Amount'].quantile(0.025))]
data = data[(data['Amount'] <= data['Amount'].quantile(0.975))]
data = data[(data['Area'] >= data['Area'].quantile(0.01))]
data = data[(data['Area'] <= data['Area'].quantile(0.99))]
data = data[data['District'] != 'unknown']
data = data[data['District'].isna() == False]
data = data[data['Seller'].isna() == False]
data = data[data['Description'].isna() == False]

In [98]:
data = data.reset_index(drop=True)

In [99]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3595 entries, 0 to 3594
Data columns (total 24 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         3595 non-null   object 
 1   City         3595 non-null   object 
 2   District     3595 non-null   object 
 3   Amount       3595 non-null   float64
 4   Currency     3595 non-null   object 
 5   Property     3595 non-null   object 
 6   Seller       3595 non-null   object 
 7   Area         3595 non-null   float64
 8   Rooms        3545 non-null   float64
 9   Bathrooms    1725 non-null   float64
 10  Parking      3595 non-null   object 
 11  Garden       3595 non-null   bool   
 12  Balcony      3595 non-null   bool   
 13  Terrace      3595 non-null   bool   
 14  Floor        3595 non-null   bool   
 15  New          3595 non-null   bool   
 16  Estate       3595 non-null   bool   
 17  Townhouse    3595 non-null   bool   
 18  Apartment    3595 non-null   bool   
 19  Land  

### Impute missing values

The next step is to fill in missing values for numeric columns `Amount` `Area` `Rooms` and `Bathrooms`. We use the KNN algorithm to accomplish this.

In [100]:
numeric = list(data.select_dtypes('number').columns)

In [101]:
mask = (data['Bathrooms'].isna() == True | data['Rooms'].isna())
missing = data[numeric]

imputer = KNNImputer(n_neighbors=5)
imputer.fit(missing)

missing = imputer.transform(missing)
missing = pd.DataFrame(missing, columns=numeric)

for col in numeric:
    data[col] = missing[col]
    
for col in numeric:
    data[col] = data[col].apply(lambda x: round(x))    

In [102]:
print(data.shape)

(3595, 24)


## Save data

Verify that there are no `NaN`s in data.

In [103]:
data.isnull().sum().sum()

0

Remove columns that will not be used further.

In [104]:
data = data.drop(['Title', 
                  'Description', 
                  'Link', 
                  'Property', 
                  'City', 
                  'Currency', 
                  'Date'], axis=1)

Take a last peek at the data.

In [105]:
data.head()

,District,Amount,Seller,Area,Rooms,Bathrooms,Parking,Garden,Balcony,Terrace,Floor,New,Estate,Townhouse,Apartment,Land,Studio
0,podgorze,850000,realtor,71,2,1,no parking,True,False,False,False,False,False,True,False,False,False
1,stare miasto,599104,realtor,65,3,1,garage,True,False,True,False,True,False,False,False,False,False
2,podgorze duchackie,450000,realtor,56,3,1,garage,False,False,True,False,False,False,False,False,False,False
3,krowodrza,339000,realtor,37,2,1,no parking,False,False,False,True,False,False,False,False,False,False
4,czyzyny,440000,realtor,41,2,1,no parking,False,True,False,True,False,True,False,False,False,False


Save it for further analysis.

In [106]:
data.to_csv('../flats-data/cleaned_data.csv', index=False)